# Water Quality Prediction — Submission Notebook

In [ ]:
!pip install uv
!uv pip install -r requirements.txt xgboost shap 

In [ ]:
import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Data manipulation and analysis
import numpy as np
import pandas as pd
from IPython.display import display

# Multi-dimensional arrays and datasets (e.g., NetCDF, Zarr)
import xarray as xr

# Geospatial raster data handling with CRS support
import rioxarray as rxr

# Raster operations and spatial windowing
import rasterio
from rasterio.windows import Window

# Feature preprocessing and data splitting
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GroupKFold, RandomizedSearchCV
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from scipy.spatial import cKDTree
from scipy.stats import uniform, randint, mstats
import shap

# Machine Learning
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, make_scorer


# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

from datetime import date
from tqdm import tqdm
import os

## 1. Load & Merge Data (join by key, not by index)

In [ ]:
wq = pd.read_csv("water_quality_training_dataset.csv")
landsat = pd.read_csv("landsat_features_training.csv")
terra = pd.read_csv("terraclimate_features_training.csv")

# Merge by key — never by index
JOIN_KEYS = ['Latitude', 'Longitude', 'Sample Date']
df = wq.merge(landsat, on=JOIN_KEYS, how='left') \
       .merge(terra, on=JOIN_KEYS, how='left')

print(f"Shape after merge: {df.shape}")
display(df.head())

## 2. Landsat Calibration (DN → Reflectance) & Recalculate Indices

In [ ]:
BANDS = ['nir', 'green', 'swir16', 'swir22']

def calibrate_landsat(data):
    """Apply Landsat C2L2 scale/offset to convert DN → surface reflectance."""
    scale, offset = 0.0000275, -0.2
    for col in BANDS:
        data[col] = data[col] * scale + offset
    # Recalculate indices from calibrated reflectance
    data['NDMI']  = (data['nir'] - data['swir16']) / (data['nir'] + data['swir16'])
    data['MNDWI'] = (data['green'] - data['swir22']) / (data['green'] + data['swir22'])
    return data

df = calibrate_landsat(df)
print("Calibrated reflectance ranges:")
for b in BANDS:
    print(f"  {b}: [{df[b].min():.4f}, {df[b].max():.4f}]")

## 3. Cloud Filter, Cluster-based Imputation & DRP Censoring

In [ ]:
# Cloud filter: after calibration, valid reflectance is roughly [0, 1.0]
for col in BANDS:
    df[col] = df[col].where(df[col].between(-0.05, 1.2), np.nan)

# Cluster-based imputation (spatial, not global median)
N_CLUSTERS = 8
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
df['geo_cluster'] = kmeans.fit_predict(df[['Latitude', 'Longitude']])

impute_cols = BANDS + ['NDMI', 'MNDWI', 'pet']
for col in impute_cols:
    cluster_med = df.groupby('geo_cluster')[col].transform('median')
    df[col] = df[col].fillna(cluster_med)
# Fallback: global median for any remaining NaN
train_medians = df[impute_cols].median()
df[impute_cols] = df[impute_cols].fillna(train_medians)

# DRP censoring: 10.0 = below detection limit → replace with DL/2
df['Dissolved Reactive Phosphorus'] = df['Dissolved Reactive Phosphorus'].replace(10.0, 5.0)

print(f"Remaining NaNs:\n{df.isna().sum()[df.isna().sum() > 0]}")

## 4. Feature Engineering

In [ ]:
dates = pd.to_datetime(df['Sample Date'], dayfirst=True)
df['month_sin'] = np.sin(2 * np.pi * dates.dt.month / 12)
df['month_cos'] = np.cos(2 * np.pi * dates.dt.month / 12)
df['year']      = dates.dt.year
df['NIR_SWIR22_ratio'] = df['nir'] / (df['swir22'] + 1e-6)
df['turbidity']        = df['swir16'] / (df['green'] + 1e-6)

# PET seasonal lags (proxy for antecedent moisture)
df = df.sort_values(['Latitude', 'Longitude', 'Sample Date']).reset_index(drop=True)
grp = df.groupby(['Latitude', 'Longitude'])['pet']
df['pet_lag1']     = grp.shift(1)
df['pet_lag2']     = grp.shift(2)
df['pet_rolling3'] = grp.transform(lambda x: x.rolling(3, min_periods=1).mean())

# Fill PET lags NaN with per-station pet (first obs has no lag)
for col in ['pet_lag1', 'pet_lag2', 'pet_rolling3']:
    df[col] = df[col].fillna(df['pet'])

FEATURE_COLS = [
    'Latitude', 'Longitude',
    'nir', 'green', 'swir16', 'swir22',
    'NDMI', 'MNDWI',
    'pet', 'pet_lag1', 'pet_lag2', 'pet_rolling3',
    'month_sin', 'month_cos', 'year',
    'NIR_SWIR22_ratio', 'turbidity',
]
print(f"Features ({len(FEATURE_COLS)}): {FEATURE_COLS}")

## 5. Winsorize Targets & Prepare Training Data

In [ ]:
TARGETS = ['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']

# Winsorize extreme outliers before log transform
for col in TARGETS:
    df[col] = mstats.winsorize(df[col], limits=[0.01, 0.01])

# Groups for spatial CV
groups = df[['Latitude', 'Longitude']].apply(lambda r: f"{r['Latitude']}_{r['Longitude']}", axis=1)

# Sample weight: upweight southern stations (test set is entirely southern)
sample_weight = np.where(df['Latitude'] < -31.0, 3.0, 1.0)

X = df[FEATURE_COLS]
y_TA_log  = np.log1p(df['Total Alkalinity'])
y_EC_log  = np.log1p(df['Electrical Conductance'])
y_DRP_log = np.log1p(df['Dissolved Reactive Phosphorus'])

print(f"Samples: {len(X)} | Unique locations: {groups.nunique()}")
print(f"Southern samples (weight=3): {(df['Latitude'] < -31.0).sum()} ({(df['Latitude'] < -31.0).mean():.1%})")

## 6. Model Training (XGBoost + GroupKFold + sample_weight, no scaler)

In [ ]:
def run_pipeline(X, y_log, groups, sample_weight, param_name, n_splits=5, n_iter=50):
    print(f"\n{'='*60}\nTraining: {param_name}\n{'='*60}")

    group_kfold = GroupKFold(n_splits=n_splits)

    # No StandardScaler — XGBoost is scale-invariant
    model = XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1, verbosity=0)

    param_dist = {
        'n_estimators':     randint(200, 800),
        'max_depth':        [4, 6, 8],
        'learning_rate':    uniform(0.05, 0.15),
        'subsample':        uniform(0.6, 0.4),
        'colsample_bytree': uniform(0.6, 0.4),
        'min_child_weight': [1, 2, 3],
        'reg_alpha':        [0, 0.01, 0.1],
        'reg_lambda':       [0.5, 1.0, 1.5],
        'gamma':            [0, 0.01, 0.05],
    }

    rmse_scorer = make_scorer(
        lambda yt, yp: np.sqrt(mean_squared_error(np.expm1(yt), np.expm1(yp))),
        greater_is_better=False)
    r2_scorer = make_scorer(
        lambda yt, yp: r2_score(np.expm1(yt), np.expm1(yp)),
        greater_is_better=True)

    search = RandomizedSearchCV(
        model, param_dist, n_iter=n_iter, cv=group_kfold,
        scoring={'r2': r2_scorer, 'rmse': rmse_scorer}, refit='r2',
        random_state=42, n_jobs=-1, verbose=1, return_train_score=True)

    search.fit(X, y_log, groups=groups, sample_weight=sample_weight)

    best = search.best_estimator_
    idx = search.best_index_
    cv = search.cv_results_

    print(f"\nBest params: { {k: v for k, v in search.best_params_.items()} }")
    print(f"CV Train R²: {cv['mean_train_r2'][idx]:.4f} | Val R²: {cv['mean_test_r2'][idx]:.4f}")
    print(f"CV Train RMSE: {-cv['mean_train_rmse'][idx]:.4f} | Val RMSE: {-cv['mean_test_rmse'][idx]:.4f}")

    return best

model_TA  = run_pipeline(X, y_TA_log,  groups, sample_weight, "Total Alkalinity")
model_EC  = run_pipeline(X, y_EC_log,  groups, sample_weight, "Electrical Conductance")
model_DRP = run_pipeline(X, y_DRP_log, groups, sample_weight, "Dissolved Reactive Phosphorus")

## 7. SHAP Feature Importance

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
for ax, model, name in zip(axes,
                           [model_TA, model_EC, model_DRP],
                           ['Total Alkalinity', 'Electrical Conductance', 'Dissolved Reactive Phosphorus']):
    explainer = shap.TreeExplainer(model)
    sv = explainer.shap_values(X)
    plt.sca(ax)
    shap.summary_plot(sv, X, feature_names=FEATURE_COLS, show=False, plot_size=None)
    ax.set_title(name, fontsize=12)
plt.tight_layout()
plt.show()

## 8. Prepare Test Set & Predict

In [ ]:
landsat_test = pd.read_csv("landsat_features_validation.csv")
terra_test   = pd.read_csv("terraclimate_features_validation.csv")
submission_template = pd.read_csv("submission_template.csv")

test = landsat_test.merge(terra_test, on=JOIN_KEYS, how='left')

# Same calibration as training
test = calibrate_landsat(test)

# Cloud filter
for col in BANDS:
    test[col] = test[col].where(test[col].between(-0.05, 1.2), np.nan)

# Impute using TRAINING cluster medians (assign test points to nearest train cluster)
test['geo_cluster'] = kmeans.predict(test[['Latitude', 'Longitude']])
for col in impute_cols:
    cluster_med = df.groupby('geo_cluster')[col].median()
    mask = test[col].isna()
    test.loc[mask, col] = test.loc[mask, 'geo_cluster'].map(cluster_med)
test[impute_cols] = test[impute_cols].fillna(train_medians)

# Feature engineering (same as training)
dates_t = pd.to_datetime(test['Sample Date'], dayfirst=True)
test['month_sin'] = np.sin(2 * np.pi * dates_t.dt.month / 12)
test['month_cos'] = np.cos(2 * np.pi * dates_t.dt.month / 12)
test['year']      = dates_t.dt.year
test['NIR_SWIR22_ratio'] = test['nir'] / (test['swir22'] + 1e-6)
test['turbidity']        = test['swir16'] / (test['green'] + 1e-6)

test = test.sort_values(['Latitude', 'Longitude', 'Sample Date']).reset_index(drop=True)
grp_t = test.groupby(['Latitude', 'Longitude'])['pet']
test['pet_lag1']     = grp_t.shift(1)
test['pet_lag2']     = grp_t.shift(2)
test['pet_rolling3'] = grp_t.transform(lambda x: x.rolling(3, min_periods=1).mean())
for col in ['pet_lag1', 'pet_lag2', 'pet_rolling3']:
    test[col] = test[col].fillna(test['pet'])

X_test = test[FEATURE_COLS]
print(f"Test shape: {X_test.shape}")

# Predict (log-space → original scale)
pred_TA  = np.expm1(model_TA.predict(X_test))
pred_EC  = np.expm1(model_EC.predict(X_test))
pred_DRP = np.expm1(model_DRP.predict(X_test))

# Align predictions back to submission_template order
test['pred_TA']  = pred_TA
test['pred_EC']  = pred_EC
test['pred_DRP'] = pred_DRP

submission_df = submission_template[JOIN_KEYS].merge(
    test[JOIN_KEYS + ['pred_TA', 'pred_EC', 'pred_DRP']], on=JOIN_KEYS, how='left'
).rename(columns={
    'pred_TA': 'Total Alkalinity',
    'pred_EC': 'Electrical Conductance',
    'pred_DRP': 'Dissolved Reactive Phosphorus',
})

display(submission_df.head())
print(f"\nNaNs in submission: {submission_df.isna().sum().sum()}")

## 9. Save & Upload Submission

In [ ]:
submission_df.to_csv("/tmp/submission.csv", index=False)
print("Saved to /tmp/submission.csv")

In [ ]:
import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

session.sql("""
    PUT file:///tmp/submission.csv
    'snow://workspace/USER$.PUBLIC."ey-hackathon"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()
print("File saved! Refresh the browser to see the files in the sidebar")